In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re
from string import Template
from tqdm.notebook import tqdm
import time

In [ ]:
#dictionary of all states and their # of districts
dict_states = {'AL': 7, 'AK': 1, 'AZ': 9, 'AR': 4, 'CA': 53, 'CO': 7, 'CT': 5, 'DE': 1,'FL': 27, 'GA': 14,
               'HI': 2, 'ID': 2, 'IL': 18, 'IN': 9, 'IA': 4, 'KS': 4, 'KY': 6, 'LA': 6, 'ME': 2, 'MD': 8,
               'MA': 9, 'MI': 13, 'MN': 8, 'MS': 4, 'MO': 8,'MT': 1, 'NE': 3, 'NV': 4, 'NH': 2, 'NJ': 12,
               'NM': 3, 'NY': 27, 'NC': 13, 'ND': 1, 'OH': 16, 'OK': 5, 'OR': 5, 'PA': 18, 'RI': 2, 'SC': 7,
               'SD': 1, 'TN': 9, 'TX': 34, 'UT': 4, 'VT': 1, 'VA': 11, 'WA': 10, 'WI': 8, 'WV': 3, 'WY': 1}                

In [ ]:
#use dictionary  to generate a list of URLs of all the states and their districts. 
def allstates_URL(state):
    """Incorporates the dictionary of states and districts into the URL to scrape the data"""
    #initialize an empty list to fill with the URLs
    URL_list = []
    
    #iterates through the dictionary and appends a list of URLs to scrape.
    for i in range(1, dict_states[state] + 1):
        URL = "https://www.opensecrets.org/races/summary?cycle=2020&id=" + state + str(i).zfill(2) + "&spec=N"
        URL_list.append(URL)
    return URL_list

In [ ]:
#empty list for all states df
all_states_df = []
for state in tqdm(dict_states):
    URL_list = allstates_URL(state)


#scrapes the data from the list of URLs and creates a data fram 
    for URL in URL_list:
        time.sleep(0.1)
        opensecrets = requests.get(URL)
        opensecrets_df = pd.read_html(opensecrets.text)
        opensecrets_df = opensecrets_df[0]
        opensecrets_df['state'] = re.findall(r"\b[A-Z]{2}\d{2}", URL)[0]
        all_states_df.append(df)

#concatenate the 
all_states = pd.concat(all_states_df)

In [ ]:
#regex for candidate, incumbent, party, and winner
candidate = r"(\w+\s\w+)"
incumbent = "Incumbent"
party = r"(\W\w\W)"
winner = "Winner"

In [ ]:
#create dictionaries for columns with multiple info in them (candidate name has party, and  )
columns_extract= {'Candidate_name': candidate,'Party':party}
columns_contains = {'Incumbent':incumbent, 'Winner':winner}

#creates columns for party, incumbent, and winner and extracts the info from the candidate column
#incorporate this in to the code above to get other columns
for key, value in columns_extract.items():
    all_states[key] = all_states['Candidate'].str.extract(value, expand=True)
    for key, value in columns_contains.items():
        all_states[key] = all_states['Candidate'].str.contains(value)

all_states = all_states[['Candidate_name', 
                                     'Party', 
                                     'Incumbent', 
                                     'Winner', 
                                     'Raised', 
                                     'Spent', 
                                     'Cash on Hand',
                                    'state']]

In [5]:
#convert object types to int
all_states['Raised'] = all_states['Raised'].str.replace('\W', '')
all_states['Spent'] = all_states['Spent'].str.replace('\W', '')
all_states['Cash on Hand'] = all_states['Cash on Hand'].str.replace('\W', '')
all_states['Raised'] = all_states['Raised'].astype(int)
all_states['Spent'] = all_states['Spent'].astype(int)
all_states['Cash on Hand'] = all_states['Cash on Hand'].astype(int)


#save dataframe as a csv for data analysis
#all_states.to_csv('../webscraping_open_secrets-purple-pumas-2/data/all_states.csv', index = 0)

  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-5-5ebe46f90b6b>:64: FutureWarning: The default value of regex will change from True to False in a future version.
  all_states['Raised'] = all_states['Raised'].str.replace('\W', '')
<ipython-input-5-5ebe46f90b6b>:65: FutureWarning: The default value of regex will change from True to False in a future version.
  all_states['Spent'] = all_states['Spent'].str.replace('\W', '')
<ipython-input-5-5ebe46f90b6b>:66: FutureWarning: The default value of regex will change from True to False in a future version.
  all_states['Cash on Hand'] = all_states['Cash on Hand'].str.replace('\W', '')


In [4]:
all_states

,Candidate_name,Party,Incumbent,Winner,Raised,Spent,Cash on Hand,state
0,Jerry Carl,(R),False,True,1971321,1859349,111973,AL01
1,James Averhart,(D),False,False,80095,78973,1122,AL01
0,Barry Moore,(R),False,True,650807,669368,13633,AL02
1,Phyllis Harvey,(D),False,False,56050,55988,0,AL02
0,Mike D,D,True,True,1193111,1218564,502234,AL03
...,...,...,...,...,...,...,...,...
0,Carol Miller,(R),True,True,869210,720796,167007,WV03
1,Hilary Turner,(D),False,False,99214,98769,446,WV03
2,Belinda Fox,(I),False,False,5207,5207,0,WV03
0,Liz Cheney,(R),True,True,3003883,3060167,153567,WY01
